In [8]:
import numpy as np
from numpy import array, zeros, diag, ones, sin, cos, tan, linspace, dot, pi
from numpy.random import uniform
from mytraj import MyTraj 
import time
from scipy import integrate
from numpy import isnan, pi, isinf
import pandas as pd
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score
import joblib
from random import gauss

# In order to use modified modules without restarting
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
def wind_shear(alt,r,p,y,vx,vy,vz,wind_direct):
    e2b = np.array([[cos(r)*cos(y), sin(p)*sin(r)*cos(y)-cos(p)*sin(y), cos(p)*sin(r)*cos(y)+sin(p)*sin(y)],
                   [cos(r)*sin(y), sin(p)*sin(r)*sin(y)+cos(p)*cos(y), cos(p)*sin(r)*sin(y)-sin(p)*cos(y)],
                   [-sin(r), sin(p)*cos(r), cos(p)*cos(r)]])
    w6 = 0.02 #reference height wind speed
    z0 = 0.6#0.045 if cat=="C" else 0.6
    W = [cos(wind_direct),sin(wind_direct),0]
    b = np.log(abs(alt)/z0)/np.log(20/z0) #reference height/z0???
    u = b*(np.dot(-w6,W))
    uw = np.dot(e2b,u)
    uw = [gauss(0,vx*0.005),gauss(0,vy*0.005),gauss(0,vz*0.005)]
    return uw

In [43]:
def model_parameters():
    g = 9.81
    m = 1.52
    Ixx, Iyy, Izz = 0.0347563, 0.0458929, 0.0977
    I1 = (Iyy - Izz) / Ixx
    I2 = (Izz - Ixx) / Iyy
    I3 = (Ixx - Iyy) / Izz
    Jr = 0.0001
    l = 0.09
    b = 8.54858e-6
    d = 1.6e-2
    
    return g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d

def model_dynamics(t, state, U):
    g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d = model_parameters()
    #states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]
        
    x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot = state
    U1, U2, U3, U4 = U
    
    if (d*U1 - 2*d*U3 - b*U4) < 0:
        omega1 = - np.sqrt(- d*U1 + 2*d*U3 + b*U4) / (2*np.sqrt(b*d))
    else:
        omega1 = - np.sqrt(d*U1 - 2*d*U3 - b*U4) / (2*np.sqrt(b*d))
        
    if (d*U1 - 2*d*U2 + b*U4) < 0:
        omega2 = -np.sqrt(-d*U1 + 2*d*U2 - b*U4) / (2*np.sqrt(b*d))
    else:
        omega2 = -np.sqrt(d*U1 - 2*d*U2 + b*U4) / (2*np.sqrt(b*d))
    
    
    if (d*U1 + 2*d*U3 - b*U4) < 0:
        omega3 = -np.sqrt(-d*U1 - 2*d*U3 + b*U4) / (2*np.sqrt(b*d))
    else:
        omega3 = -np.sqrt(d*U1 + 2*d*U3 - b*U4) / (2*np.sqrt(b*d))
        
    if (d*U1 + 2*d*U2 + b*U4) < 0:
        omega4 = -np.sqrt(-d*U1 - 2*d*U2 - b*U4) / (2*np.sqrt(b*d))
    else:
        omega4 = -np.sqrt(d*U1 + 2*d*U2 + b*U4) / (2*np.sqrt(b*d))
        
    
    omega = (omega2 + omega4 - omega1 - omega3)
    
    state_dot = np.zeros(12)
    state_dot[0] = x_dot
    state_dot[1] = y_dot
    state_dot[2] = z_dot
    state_dot[3] = phi_dot
    state_dot[4] = theta_dot
    state_dot[5] = psi_dot
    
    state_dot[6] = (cos(phi)*sin(theta)*cos(psi) + sin(phi)*sin(psi))*U[0]/m
    state_dot[7] = (cos(phi)*sin(theta)*sin(psi) - sin(phi)*cos(psi))*U[0]/m
    state_dot[8] = -g + cos(phi)*cos(theta)*U[0]/m    
    state_dot[9] = theta_dot*psi_dot*I1 - Jr / Ixx * theta_dot * omega  + l/Ixx*U[1]
    state_dot[10] = phi_dot*psi_dot*I2 + Jr / Iyy * phi_dot * omega + l/Iyy*U[2]
    state_dot[11] = phi_dot*theta_dot*I3 + 1/Izz*U[3]
    
    return state_dot
    
def backstepping(A1, A2, A3, A4, A5, A6, U_list, ref_traj, states):
    g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d = model_parameters()
    
    U1, U2, U3, U4 = U_list
    
    #states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]
    x, y, z = states[0], states[1], states[2]
    phi, theta, psi = states[3], states[4], states[5]
    x_dot, y_dot, z_dot = states[6], states[7], states[8]
    phi_dot, theta_dot, psi_dot = states[9], states[10], states[11]
    
#     ref_traj = [xd[i], yd[i], zd[i], xd_dot[i], yd_dot[i], zd_dot[i], 
#                 xd_ddot[i], yd_ddot[i], zd_ddot[i], xd_dddot[i], yd_dddot[i],
#                 xd_ddddot[i], yd_ddddot[i], psid[i], psid_dot[i], psid_ddot[i]]

    
    xd, yd, zd = ref_traj[0], ref_traj[1], ref_traj[2], 
    xd_dot, yd_dot, zd_dot = ref_traj[3], ref_traj[4], ref_traj[5]
    xd_ddot, yd_ddot, zd_ddot = ref_traj[6], ref_traj[7], ref_traj[8]
    xd_dddot, yd_dddot = ref_traj[9], ref_traj[10]
    xd_ddddot, yd_ddddot = ref_traj[11], ref_traj[12]
    psid, psid_dot, psid_ddot = ref_traj[13], ref_traj[14], ref_traj[15]
    
    x1, x2, x3 = array([[x], [y]]), array([[x_dot], [y_dot]]), array([[phi], [theta]])
    x4, x5, x6 = array([[phi_dot], [theta_dot]]), array([[psi], [z]]), array([[psi_dot], [z_dot]])
    
    g0 = array([[np.cos(psi), np.sin(psi)],  [np.sin(psi), -np.cos(psi)]])
    g0_inv = array([[np.cos(psi), np.sin(psi)],  [np.sin(psi), -np.cos(psi)]])
    
    g1 = array([[theta_dot*psi_dot*I1],  [phi_dot*psi_dot*I2]])
    g2 = array([[phi_dot*theta_dot*I3],  [-g]])
    
    l0 = array([[np.cos(phi)*np.sin(theta)],  [np.sin(phi)]])*U1/m 
    dl0_dx3 = array([[-np.sin(phi)*np.sin(theta), np.cos(phi)*np.cos(theta)],  [np.cos(phi), 0]])*U1/m 
    dl0_dx3_inv = array([[0, 1/np.cos(phi)],  [1/np.cos(theta)*1/np.cos(phi), 1/np.cos(phi)*np.tan(theta)*np.tan(phi)]])*m/U1 
    dl0_dx3_inv_dot = array([[0, 1/np.cos(phi)*np.tan(phi)*phi_dot], 
                             [1/np.cos(theta)*1/np.cos(phi)*(np.tan(theta)*theta_dot + np.tan(phi)*phi_dot), 1/np.cos(phi)*((1/np.cos(theta))**2*np.tan(phi)*theta_dot + (-1+2*(1/np.cos(phi))**2)*np.tan(theta)*phi_dot)]])*m/U1 
                
#     Omega_square = Omega_coef_inv * abs([U1/b  U2/b  U3/b  U4/d]) 
#     Omega_param = sqrt(Omega_square) 
#     omega = Omega_param(2) + Omega_param[3] - Omega_param(1) - Omega_param(3) 

#     h1 = [-Jr/Ixx*theta_dot*omega  Jr/Iyy*phi_dot*omega] 
    h1 = 0 
    k1 = diag([l/Ixx, l/Iyy]) 
    k1_inv = diag([Ixx/l, Iyy/l]) 
    k2 = diag([1/Izz, np.cos(phi)*np.cos(theta)/m]) 
    k2_inv = diag([Izz, m/(np.cos(phi)*np.cos(theta))]) 
    
    x1d = array([[xd], [yd]])  
    x1d_dot = array([[xd_dot], [yd_dot]]) 
    x1d_ddot = array([[xd_ddot], [yd_ddot]]) 
    x1d_dddot = array([[xd_dddot], [yd_dddot]]) 
    x1d_ddddot = array([[xd_ddddot], [yd_ddddot]]) 
    
    x5d = array([[psid], [zd]])
    x5d_dot = array([[psid_dot], [zd_dot]]) 
    x5d_ddot = array([[psid_ddot], [zd_ddot]]) 
    
    z1 = x1d - x1 
    v1 = x1d_dot + dot(A1,z1) 
    z2 = v1 - x2 
    z1_dot = -dot(A1,z1) + z2 
    v1_dot = x1d_ddot + dot(A1,z1_dot) 
    v2 = dot(g0_inv, z1 + v1_dot + dot(A2,z2)) 
    z3 = v2 - l0  
    z2_dot = -z1 - dot(A2,z2) + dot(g0,z3) 
    z1_ddot = -dot(A1,z1_dot) + z2_dot 
    v1_ddot = x1d_dddot + dot(A1, z1_ddot) 
    v2_dot = dot(g0_inv, z1_dot + v1_ddot + dot(A2,z2_dot)) 
    v3 = dot(dl0_dx3_inv, dot(g0.T,z2) + v2_dot + dot(A3, z3)) 
    z4 = v3 - x4 
    z3_dot = -dot(g0.T, z2) - dot(A3,z3) + dot(dl0_dx3, z4) 
    z2_ddot = - z1_dot - dot(A2, z2_dot) + dot(g0, z3_dot) 
    z1_dddot = -dot(A1, z1_ddot) + z2_ddot 
    v1_dddot = x1d_ddddot + dot(A1, z1_dddot) 
    v2_ddot = dot(g0_inv, z1_ddot + v1_dddot + dot(A2, z2_ddot)) 
    v3_dot = dot(dl0_dx3_inv, dot(g0.T, z2_dot) + v2_ddot + dot(A3, z3_dot)) + dot(dl0_dx3_inv_dot, dot(g0.T, z2) + v2_dot + dot(A3, z3))
    l1 = dot(k1_inv, dot(dl0_dx3.T, z3) + v3_dot - g1 - h1 + dot(A4, z4)).ravel()
    
    z5 = x5d - x5 
    v5 = x5d_dot + dot(A5, z5) 
    z6 = v5 - x6 
    z5_dot = - dot(A5, z5) + z6 
    v5_dot = x5d_ddot + dot(A5, z5_dot) 
    l2 = dot(k2_inv, z5 + v5_dot - g2 + dot(A6, z6)).ravel()
    
    U1, U2, U3, U4 = l2[1], l1[0], l1[1], l2[0]
    
    U1 = np.clip(U1, 1.0, 1e2)
    U2 = np.clip(U2, -1e3, 1e3)
    U3 = np.clip(U3, -1e3, 1e3)
    U4 = np.clip(U4, -1e2, 1e2)
    
    U = np.array([U1, U2, U3, U4])
    
    return U

def get_control_input(cont, Controllers, U0, current_traj, state):
    #single_shot = ["Back"] # just a check flight to test the coefficients
    if (cont == Controllers[0]): #Backstepping_1
        A1, A2, A3 = 15*diag([1,1]), 10*diag([1,1]), 15*diag([1,1]) 
        A4, A5, A6 = 10*diag([1,1]), 15*diag([1,1]), 10*diag([1,1]) 
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state) 
    elif (cont == Controllers[1]): #Backstepping_2
        A1, A2, A3 = 10*diag([1,1]), 5*diag([1,1]), 10*diag([1,1]) 
        A4, A5, A6 = 5*diag([1,1]), 10*diag([1,1]), 5*diag([1,1])
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state) 
    elif (cont == Controllers[2]): #Backstepping_3
        A1, A2, A3 = 5*diag([1,1]), 3*diag([1,1]), 10*diag([1,1]) 
        A4, A5, A6 = 7*diag([1,1]), 1*diag([1,1]), 1*diag([1,1])  
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state)
    elif (cont == Controllers[3]): #Backstepping_4
        A1, A2, A3 = 2*diag([1,1]), 5*diag([1,1]), 2*diag([1,1]) 
        A4, A5, A6 = 5*diag([1,1]), 2*diag([1,1]), 5*diag([1,1]) 
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state)
    elif (cont == single_shot[0]): #Single Shot
        A1, A2, A3 = 1*diag([1,1]), 1*diag([1,1]), 1*diag([1,1]) 
        A4, A5, A6 = 1*diag([1,1]), 1*diag([1,1]), 1*diag([1,1]) 
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state)
    return U

In [44]:
def predict(X, model, device, method="MAX"):
    softmax = nn.Softmax(dim=1)
    inputs = torch.from_numpy(X).to(device)
    outputs = model(inputs.float())
 
    if method=="MAX":
        _, pred = torch.max(outputs, 1)
        return pred[0]
    elif method=="DICE":
        probs = softmax(outputs).cpu().detach().numpy()[0]
        pred_index = np.random.choice([0, 1, 2, 3], 1, p=probs)[0]
        return pred_index
    elif method=="MIX":
        probs = softmax(outputs).cpu().detach().numpy()[0]     
        return probs
        
    
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop_layer = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(37, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc3(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc4(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc5(x))
        x = self.drop_layer(x)
        x = self.fc6(x)
        return x

In [45]:
def write_stats(stats_columns, stats, filename): #testno,stats_columns
    df_stats = pd.DataFrame([stats], columns=stats_columns)
    df_stats.to_csv(filename, mode='a', index=False,header=not os.path.isfile(filename))
    
def quad_sim(Tf, state0, ref_traj, scaler=None, model=None, device=None, status="TEST", method="MAX"):
#     states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]

    N = ref_traj.shape[0]
    costValue = 0
    U = [1,0,0,0]

    Controllers = ["Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
    single_shot = ["Back"] # just a check flight to test the coefficients
    count_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
    U_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
    state = np.copy(state0)
    
    for i in range(N):
        
        #     ref_traj = [xd[i], yd[i], zd[i], xd_dot[i], yd_dot[i], zd_dot[i], 
        #                 xd_ddot[i], yd_ddot[i], zd_ddot[i], xd_dddot[i], yd_dddot[i],
        #                 xd_ddddot[i], yd_ddddot[i], psid[i], psid_dot[i], psid_ddot[i]]
        prev_traj_index = np.maximum(0, i-1)
        
        current_traj = [ref_traj[i][0], ref_traj[i][1], ref_traj[i][2], ref_traj[i][3], ref_traj[i][4], ref_traj[i][5], 
                        ref_traj[i][6], ref_traj[i][7], ref_traj[i][8], ref_traj[i][9], ref_traj[i][10], ref_traj[i][11],
                        ref_traj[i][12], ref_traj[i][13], ref_traj[i][14], ref_traj[i][15]]
        
        prev_traj = [ref_traj[prev_traj_index][0], ref_traj[prev_traj_index][1], ref_traj[prev_traj_index][2], ref_traj[prev_traj_index][3], ref_traj[prev_traj_index][4], ref_traj[prev_traj_index][5], 
                     ref_traj[prev_traj_index][6], ref_traj[prev_traj_index][7], ref_traj[prev_traj_index][8], ref_traj[prev_traj_index][9], ref_traj[prev_traj_index][10], ref_traj[prev_traj_index][11],
                     ref_traj[prev_traj_index][12], ref_traj[prev_traj_index][13], ref_traj[prev_traj_index][14], ref_traj[prev_traj_index][15]]

        U0 = np.copy(U)
        
        t_current = Tf / N * i
        time_rate = float(t_current / Tf)
        Upr_abs_sum = np.sum(np.abs(U0))
        
#         feature_names = ['x0', 'y0', 'z0', 'x_dot0','y_dot0','z_dot0', 'phi0','theta0','yaw0', 'phi_dot0','theta_dot0','yaw_dot0', 
#                  'xf', 'yf', 'zf', 'x_dotf','y_dotf','z_dotf','x_ddotf','y_ddotf','z_ddotf',
#                  'pos_diffx','pos_diffy','pos_diffz','time_rate','t', 'Tf', 
#                  'xp', 'yp', 'zp', 'x_dotp','y_dotp','z_dotp','x_ddotp','y_ddotp','z_ddotp', 'u_abs_p', 'wind_direct']
        
        #WIND EFFECT    
        wind_direct = (np.random.randint(1,360)/180)*pi #Deg2Rad
        wind_noisev = wind_shear(state[2],state[3],state[4],state[5],state[6],state[7],state[8],wind_direct)
        
        #Test part
        if status == "TEST":
            ## ADD DISTURBANCE ##
            state[6] += wind_noisev[0]
            state[7] += wind_noisev[1]
            state[8] += wind_noisev[2]
            
            X_test = np.array([state[0], state[1], state[2], state[6], state[7], state[8], state[3], state[4], state[5], state[9], state[10], state[11],
                        current_traj[0], current_traj[1], current_traj[2], current_traj[3], current_traj[4], current_traj[5], current_traj[6], current_traj[7], current_traj[8],
                        state[0]-current_traj[0], state[1]-current_traj[1], state[2]-current_traj[2], time_rate, t_current, Tf, 
                        prev_traj[0], prev_traj[1], prev_traj[2], prev_traj[3], prev_traj[4], prev_traj[5], prev_traj[6], prev_traj[7], prev_traj[8], 
                        Upr_abs_sum, wind_direct]).reshape(1,-1)
            X_test = scaler.transform(X_test)
        
            if method == "MAX":
                pred = predict(X_test, model, device, method)
                cont = Controllers[pred]
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                count_dict[cont] += 1
                
            elif method == "DICE":
                pred = predict(X_test, model, device, method)
                cont = Controllers[pred]
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                count_dict[cont] += 1
                
            elif method == "MIX":
                probs = predict(X_test, model, device, method)
                U = 0
                for k in range(4):
                    cont = Controllers[k]
                    cont_prob = probs[k]
                    U_single = get_control_input(cont, Controllers, U0, current_traj, state)
                    U = U + (cont_prob * U_single)
                    
            elif method == "DT":
                pred = decision_tree.predict(X_test)[0]
                cont = Controllers[pred]
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                count_dict[cont] += 1
                
            elif method == "FOREST":
                pred = random_forest.predict(X_test)[0]
                cont = Controllers[pred]
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                count_dict[cont] += 1
            else:
                #This time a controller info comes in method variable
                U = get_control_input(method, Controllers, U0, current_traj, state)
                
            sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, 1e-3), y0=state, args = (U,))
            state = sol.y[:,-1]
            
            
            if (np.abs(state[3]) > pi/2)  | (np.abs(state[4]) > pi/2):
                costValue = 1e6
                method_cost_list[method].append(costValue)
                break
            else:
                position_tracking_error = np.power((current_traj[0]-state[0]),2) + np.power((current_traj[1]-state[1]),2) + np.power((current_traj[2]-state[2]),2)
                velocity_tracking_error = np.power((current_traj[3]-state[6]),2) + np.power((current_traj[4]-state[7]),2) + np.power((current_traj[5]-state[8]),2)
                angular_error = state[3]**2 + state[4]**2 + state[5]**2
                cont_input = U[0]**2 + U[1]**2 + U[2]**2 + U[3]**2
                costValue = costValue + (position_tracking_error +  .25*angular_error)
                
            
            
        elif status == "DATA_COLLECTION":
            cost_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
            state_dict = {"Backstepping_1": [], "Backstepping_2": [], "Backstepping_3": [], "Backstepping_4": []}
            
            for cont in Controllers:  
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                noisy_state = state
                ## ADD DISTURBANCE ##
                noisy_state[6] += wind_noisev[0]
                noisy_state[7] += wind_noisev[1]
                noisy_state[8] += wind_noisev[2]
                
                sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, 1e-3), y0=noisy_state, args = (U,))
                new_state = sol.y[:,-1]
                
                state_dict[cont] = new_state
                U_dict[cont] = U
                
                if (np.abs(new_state[3]) > pi/2)  | (np.abs(new_state[4]) > pi/2):
                    costValue = 1e12
                    cost_dict[cont] = costValue
                    continue

            #     states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]

                position_tracking_error = np.power((current_traj[0]-new_state[0]),2) + np.power((current_traj[1]-new_state[1]),2) + np.power((current_traj[2]-new_state[2]),2)
                velocity_tracking_error = np.power((current_traj[3]-new_state[6]),2) + np.power((current_traj[4]-new_state[7]),2) + np.power((current_traj[5]-new_state[8]),2)
                angular_error = new_state[3]**2 + new_state[4]**2 + new_state[5]**2
                cont_input = U[0]**2 + U[1]**2 + U[2]**2 + U[3]**2
                costValue = (position_tracking_error +  .25*angular_error)
                cost_dict[cont] = costValue
                
                
            min_cost_index = min(cost_dict.items(), key=lambda x: x[1])[0]
            count_dict[min_cost_index] += 1   
            U = U_dict[min_cost_index]
            
            state = state_dict[min_cost_index]
            
            if(i%20==0):
                write_stats(flight_columns,
                        [state[0], state[1], state[2], state[6], state[7], state[8], state[3], state[4], state[5], state[9], state[10], state[11],
                        current_traj[0], current_traj[1], current_traj[2], current_traj[3], current_traj[4], current_traj[5], current_traj[6], current_traj[7], current_traj[8],
                        state[0]-current_traj[0], state[1]-current_traj[1], state[2]-current_traj[2], time_rate, t_current, Tf, 
                        prev_traj[0], prev_traj[1], prev_traj[2], prev_traj[3], prev_traj[4], prev_traj[5], prev_traj[6], prev_traj[7], prev_traj[8], 
                        Upr_abs_sum, wind_direct, min_cost_index], flight_filename)
            
        elif status == "SINGLE_SHOT":
            ## ADD DISTURBANCE ##
            state[6] += wind_noisev[0]
            state[7] += wind_noisev[1]
            state[8] += wind_noisev[2]
            
            U = get_control_input(single_shot[0],Controllers, U0, current_traj, state)
            sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, 1e-3), y0=state, args = (U,))
            state = sol.y[:,-1]            

            if (np.abs(state[3]) > pi/2)  | (np.abs(state[4]) > pi/2):
                costValue = 1e12
                break

            position_tracking_error = np.power((current_traj[0]-state[0]),2) + np.power((current_traj[1]-state[1]),2) + np.power((current_traj[2]-state[2]),2)
            velocity_tracking_error = np.power((current_traj[3]-state[6]),2) + np.power((current_traj[4]-state[7]),2) + np.power((current_traj[5]-state[8]),2)
            angular_error = state[3]**2 + state[4]**2 + state[5]**2
            cont_input = U[0]**2 + U[1]**2 + U[2]**2 + U[3]**2
            costValue = costValue + (position_tracking_error +  .25*angular_error)
            
        

    if (status == "TEST" or status == "SINGLE_SHOT"):
        print ("Cost Value: ", costValue)
        method_cost_list[method].append(costValue)
    if (method == "MAX" or method == "DICE" or method == "DT" or method == "FOREST"):
        print ("How many times Backstepping_1 is called?: ", count_dict["Backstepping_1"])
        print ("How many times Backstepping_2 is called?: ", count_dict["Backstepping_2"])
        print ("How many times Backstepping_3 is called?: ", count_dict["Backstepping_3"])
        print ("How many times Backstepping_4 is called?: ", count_dict["Backstepping_4"])
    print ("Final state, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3}, vx: {6:.3}, vy: {7:.3}, vz: {8:.3}, p: {9:.3}, q: {10:.3}, r: {11:.3}".format(state[0],state[1],state[2],state[3],state[4],state[5], state[6],state[7],state[8], state[9],state[10],state[11]))

In [ ]:
flight_columns = ['x0', 'y0', 'z0', 'x_dot0','y_dot0','z_dot0', 'phi0','theta0','yaw0', 'phi_dot0','theta_dot0','yaw_dot0', 
                 'xf', 'yf', 'zf', 'x_dotf','y_dotf','z_dotf','x_ddotf','y_ddotf','z_ddotf',
                 'pos_diffx','pos_diffy','pos_diffz','time_rate','t', 'Tf', 
                 'xp', 'yp', 'zp', 'x_dotp','y_dotp','z_dotp','x_ddotp','y_ddotp','z_ddotp', 'u_abs_p', 'wind_direct',
                 'controller_ID']

cost_columns = ["MAX", "MIX", "DICE", "DT", "FOREST", "Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
flight_filename = "flight_gauss.csv"
cost_filename = 'method_cost_list.csv'

STATUS = "DATA_COLLECTION"
#STATUS = "SINGLE_SHOT"
#STATUS = "TEST"
METHOD_LIST = ["MAX", "MIX", "DICE", "DT", "FOREST", "Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
method_cost_list = {"MAX":[], "MIX":[], "DICE":[], "DT":[], "FOREST":[], "Backstepping_1":[], "Backstepping_2":[], "Backstepping_3":[], "Backstepping_4":[]}
dtau = 1e-3

if STATUS == "TEST":        
    #Decision Tree classifier
    decision_tree = joblib.load('Classifier/decision_tree.sav')
    #Random forest classifier
    random_forest = joblib.load('Classifier/forest.sav')
    #To normalize data
    scaler = joblib.load('Classifier/std_scaler.bin')

    #Neural network
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Net()
    model = model.to(device)
    model.load_state_dict(torch.load('Classifier/best_model.pt'))
    model.eval()


K = 150 # how many different trajectories to be followed

v_lim = 5.0
acc_lim = 2.5
ang_lim = 75*np.pi/180
ang_vel_lim = 1.0
pos_lim = 15

U_list = []

print ("RUN MODE: ", STATUS)

for j in range(K):
    pos0 = [uniform(low=-2.0, high=2.0), uniform(low=-2.0, high=2.0), uniform(low=0.0, high=10.0)]
    vel0 = [uniform(low=-v_lim, high=v_lim), uniform(low=-v_lim, high=v_lim), uniform(low=-v_lim, high=v_lim)]
    acc0 = [uniform(low=-acc_lim, high=acc_lim), uniform(low=-acc_lim, high=acc_lim), uniform(low=-acc_lim, high=acc_lim)]
    ang0 = [uniform(low=-ang_lim, high=ang_lim), uniform(low=-ang_lim, high=ang_lim), uniform(low=-1.57, high=1.57)]
    ang_vel0 = [uniform(low=-ang_vel_lim, high=ang_vel_lim), uniform(low=-ang_vel_lim, high=ang_vel_lim), uniform(low=-ang_vel_lim, high=ang_vel_lim)]
    posf = [uniform(low=-pos_lim, high=pos_lim), uniform(low=-pos_lim, high=pos_lim), uniform(low=0.0, high=1.5*pos_lim)]
    velf = [0.,0.,0.]#[uniform(low=0, high=0.5), uniform(low=0, high=0.5), uniform(low=0, high=0.5)]
    accf = [0.,0.,0.]#[uniform(low=0, high=0.15), uniform(low=0, high=0.15), uniform(low=0, high=0.15)]
    Tf = uniform(low=5, high=15)
    
    traj = MyTraj(gravity = -9.81)
    trajectory = traj.givemetraj(pos0, vel0, acc0, posf, velf, accf, Tf)
    
    N = int(Tf/dtau)
    t = linspace(0,Tf,num = N)
    
    xd, yd, zd, psid = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_dot, yd_dot, zd_dot, psid_dot = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_ddot, yd_ddot, zd_ddot, psid_ddot = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_dddot, yd_dddot, zd_dddot = zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_ddddot, yd_ddddot, zd_ddddot = zeros(t.shape), zeros(t.shape), zeros(t.shape)
    
    i = 0
    ts = 0
    
    for i in range(N):
        pos_des, vel_des, acc_des = traj.givemepoint(trajectory, ts)
        
        xd[i], yd[i], zd[i] = pos_des[0], pos_des[1], pos_des[2]
        xd_dot[i], yd_dot[i], zd_dot[i] = vel_des[0], vel_des[1], vel_des[2]
        xd_ddot[i], yd_ddot[i], zd_ddot[i] = acc_des[0], acc_des[1], acc_des[2]
        ts += dtau
    
    
    ref_traj = np.c_[xd, yd, zd, xd_dot, yd_dot, zd_dot, xd_ddot, yd_ddot, zd_ddot,
                     xd_dddot, yd_dddot, xd_ddddot, yd_ddddot,
                     psid, psid_dot, psid_ddot]
    
    state0 = np.array([pos0[0], pos0[1], pos0[2], ang0[0], ang0[1], ang0[2], 
                       vel0[0], vel0[1], vel0[2], ang_vel0[0], ang_vel0[1], ang_vel0[2]])
    
        
    
    print ("")
    print ("-"*120)
    print ("Init, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3}, vx: {6:.3}, vy: {7:.3}, vz: {8:.3}, p: {9:.3}, q: {10:.3}, r: {11:.3}".format(pos0[0], pos0[1], pos0[2], ang0[0], ang0[1], ang0[2], vel0[0], vel0[1], vel0[2], ang_vel0[0], ang_vel0[1], ang_vel0[2]))
    print ("Goal, x: {0:.3}, y: {1:.3}, z: {2:.3}, vx: {3:.3}, vy: {4:.3}, vz: {5:.3} in {6:.3} s.".format(posf[0], posf[1], posf[2], velf[0], velf[1], velf[2], Tf))
    
    if STATUS == "DATA_COLLECTION" or STATUS == "SINGLE_SHOT":
        quad_sim(Tf, state0, ref_traj, status=STATUS)
    elif STATUS == "TEST":
        for method in METHOD_LIST:
            print ("\nTEST MODE: ", method)
            quad_sim(Tf, state0, ref_traj, scaler=scaler, model=model, device=device, status=STATUS, method=method)

        if j % 1 == 0:
            write_stats(cost_columns,
                        [method_cost_list["MAX"], method_cost_list["MIX"], method_cost_list["DICE"], method_cost_list["DT"],
                         method_cost_list["FOREST"], method_cost_list["Backstepping_1"], method_cost_list["Backstepping_2"], 
                         method_cost_list["Backstepping_3"], method_cost_list["Backstepping_4"]], cost_filename)

RUN MODE:  DATA_COLLECTION

------------------------------------------------------------------------------------------------------------------------
Init, x: 0.223, y: -0.785, z: 7.02, phi: -0.397, theta: -0.347, psi: -0.619, vx: -3.45, vy: 1.13, vz: -2.04, p: 0.333, q: 0.121, r: -0.459
Goal, x: -8.05, y: -13.6, z: 7.96, vx: 0.0, vy: 0.0, vz: 0.0 in 5.05 s.
How many times Backstepping_1 is called?:  2274
How many times Backstepping_2 is called?:  212
How many times Backstepping_3 is called?:  1469
How many times Backstepping_4 is called?:  1096
Final state, x: -8.05, y: -13.6, z: 7.96, phi: -0.00686, theta: 0.00215, psi: 2.09e-07, vx: -0.00506, vy: 0.00132, vz: -0.00151, p: 0.704, q: -0.0119, r: -1.83e-06

------------------------------------------------------------------------------------------------------------------------
Init, x: 1.81, y: -1.22, z: 7.18, phi: -0.522, theta: -0.529, psi: 1.26, vx: -2.4, vy: -0.741, vz: -1.84, p: 0.901, q: -0.177, r: -0.738
Goal, x: -5.48, y: 4.32, z